In [2]:
from ex2_utils import create_epanechnik_kernel, generate_responses_1, get_patch
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math

In [11]:
def mean_shift(frame, kernel_size, position, eps=0.05):

    if kernel_size % 2 == 0:
        kernel_size = kernel_size + 1
        print("kernel_size must me an odd number, switching to kernel_size", kernel_size)

    xi_X = np.array([list(range(-math.floor(kernel_size / 2), math.floor(kernel_size / 2) + 1)) for i in range(kernel_size)])
    xi_Y = xi_X.T

    positions = []
    iter_count = 0
    while True:
        wi, _ = get_patch(frame, position, (kernel_size, kernel_size))

        # Calculate the changes in both x and y directions
        xk_X = np.sum(xi_X * wi) / np.sum(wi)
        xk_Y = np.sum(xi_Y * wi) / np.sum(wi)

        # Update the position accordingly
        position = (position[0] + xk_X, position[1] + xk_Y)
        positions.append(tuple(map(int, np.floor(position))))
        print(position)

        # Check if the algorithm converged
        if abs(xk_X) < eps and abs(xk_Y) < eps:
            break

        # Sanity check if the algorihtm does not converge
        if iter_count >= 500:
            break

        iter_count += 1

    position = tuple(map(int, np.floor(position)))
    return position, iter_count, positions
